# Basic energy minimization in mBuild

This notebook provides an overview of performing energy minimization in mBuild.

In [1]:
import mbuild as mb
import numpy as np
import warnings
warnings.filterwarnings('ignore') 


from mbuild.lib.recipes.alkane import Alkane

## Default energy minimization

After initializing an mBuild `Compound`, we can perform energy minimization by simply calling the `energy_minimize` function as part of the `Compound` class.  By default, mBuild will attempt to call [OpenBabel](http://openbabel.org) to perform the energy minimization. Note, [OpenMM](http://openmm.org) is also supported but requires a force field XML file to be specified.

In the example below, we will initialize a hexane molecule and perform energy minimization using OpenBabel. In the xample below, We willclone of the hexane molecule (i.e., make a copy) before energy minimization to allow us view the pre- and post- minimized structure side-by-side. 

In [2]:
hexane = Alkane(n=6)
hexane_pre_emin = mb.clone(hexane)
hexane.energy_minimize()

'''
The following code snippet will enable us to visualize 
the pre- and post-minimized structures side-by-side.
This is done by create an empty mbuild compound, system, 
and adding the pre- and post- minimized structures to it.
To ensure the molecules are not sitting on top of each other,
we will translate the center-of-mass of each.
'''

# shift the two molecules so they are not on top of each other
hexane_pre_emin.translate([-0.5,0,0])
hexane.translate([0.5,0,0])

#create an empty compound
#add the two molecules
system = mb.Compound()
system.add(hexane_pre_emin)
system.add(hexane)

#visualize the system
system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Preserving the center-of-mass location
By default, the energy minimization routine will shift the `Compound` such that the center-of-mass is unchanged. This will apply regardless of the energy minimization library used (i.e., OpenBabel or OpenMM).

Ensuring the center-of-mass is unchanged provides consistency and may be needed for certain systems/methods.  While this shifting could be easily done by a user, it would require a user to save the center-of-mass before energy minimization is called, since the coordinates in the Compound are updated by the call (rather than returning a copy of the Compound with modified coordinates).

The following code is essentially the same as above, but here we explicitly check to see if the center-of-mass position is unchanged.

In [3]:
hexane = Alkane(n=6)
initial_com = hexane.pos

hexane.energy_minimize()
final_com = hexane.pos

#calculate the Euclidian distance between the two vectors
diff = np.linalg.norm(initial_com-final_com)
print("Net change in COM: {0:.10f} nm ".format(diff))

if np.allclose(initial_com, final_com, rtol=1e-10, atol=1e-10):
    print("COM position is effectively unchanged.")
else:
    print("COM position has changed after energy minimization.")

Net change in COM: 0.0000000000 nm 
COM position is effectively unchanged.


To disable shifting of the center-of-mass, we can simple setting the `shift_com` argument to `False`, as done in the code below. 

In [4]:
hexane = Alkane(n=6)
initial_com = hexane.pos

hexane.energy_minimize(shift_com=False)
final_com = hexane.pos

#calculate the Euclidian distance between the two vectors
diff = np.linalg.norm(initial_com-final_com)
print("Net change in COM: {0:.10f} nm ".format(diff))

if np.allclose(initial_com, final_com, rtol=1e-10, atol=1e-10):
    print("COM position is effectively unchanged.")
else:
    print("COM position has changed after energy minimization.")



Net change in COM: 0.0089574002 nm 
COM position has changed after energy minimization.


Note, the change in the center-of-mass position is very small but still non-zero. The small deviation is most likely due to the relatively small size and low complexity of the molecule. 

### Preserving the center-of-mass location of a user specified sub-Compound

Rather than shifting by the center-of-mass of the whole molecule, we can specify this shifting based upon a child of the `Compound` by specifying passing this to the `anchor` argument. 

Note, just like center-of-mass shifting, this simply shifts the energy minimized `Compound` after the fact; it does not impose any sort of constraints on the optimization itself.  

In the following code, we will specify the anchor to be the terminal methyl group of the hexane chain. 

In [5]:
hexane = Alkane(n=6)
anchor_point = hexane.labels['chain'].labels['CH3'][0]

initial_com = anchor_point.pos

hexane.energy_minimize(shift_com=False, anchor=anchor_point)

final_com = anchor_point.pos

diff = np.linalg.norm(initial_com-final_com)
print("Net change in anchor COM: {0:.10f} nm ".format(diff))


if np.allclose(initial_com, final_com, rtol=1e-10, atol=1e-10):
    print("COM position of the anchor is effectively unchanged.")
else:
    print("COM position of the anchor has changed after energy minimization.")

Net change in anchor COM: 0.0000000000 nm 
COM position of the anchor is effectively unchanged.


Since mBuild is flexible and can perform the same basic operations on any `Compound`, the user specified anchor could alternatively be a `Compound` that represents a point particle (for example, the C atom in the methyl group).

Note, the code will check to ensure that the specified anchor is part of the larger `Compound`.  For example, in the code below, we will set the anchor to be an arbitrary point not part of the hexane molecule. This will produce an error by design.

In [6]:
hexane = Alkane(n=6)
anchor_point = mb.Compound(pos=[0,0,0], name='O')
hexane.energy_minimize(shift_com=False, anchor=anchor_point)


MBuildError: Anchor: <O pos=([0. 0. 0.]), 0 bonds, id: 5488829152> is not part of the Compound: <Alkane 20 particles, 19 bonds, non-periodic, id: 5488829536>that you are trying to energy minimize.